In [ ]:
import os
import uuid, re

from paddleocr import PaddleOCR
import albumentations as A
import cv2
import numpy as np
import pandas as pd
from PIL import Image


In [ ]:
## Image Pre-processing
transform = A.Compose([
    #A.Resize(height = 512, width = 512),
    A.RandomResizedCrop(p=1, height= 820 ,width= 820, scale=(0.4, 0.75),ratio=(0.40, 1.10)),
    A.RandomBrightnessContrast(always_apply=True, p=1.0, brightness_limit=(0, 0.3), contrast_limit=(-0.2, 0.3), brightness_by_max=False),
    A.SafeRotate(always_apply=False, p=0.5, limit=(-10, 30), interpolation=1, border_mode=0, value=None, mask_value=None),
    A.CLAHE(always_apply=False, p=0.5, clip_limit=(1, 15), tile_grid_size=(8, 8)),
    #A.HorizontalFlip(always_apply=False, p=0.5),
    A.CenterCrop(always_apply=True, p=1, height=512, width=512)
    #A.pytorch.transforms.ToTensorV2()
])
transform_o = A.Compose([
    A.Resize(height = 400, width = 256)
])

In [ ]:
# 난수, 경로 파라미터
unique_filename = str(uuid.uuid4())
img_path = ''
save_path = ''
max_text_length= 2
drop_score = 0.9

In [ ]:
# 이미지 전처리
im_bgr = cv2.imread(img_path)
img = im_bgr[:, :, ::-1]
preprocessed_img = transform(image=img)["image"]


In [ ]:
# 전처리한 이미지 저장
img_array = Image.fromarray(preprocessed_img.astype(np.uint8))
file_name = f"{unique_filename}_preprocessed.jpg"
img_array.save(os.path.join(save_path,file_name))

In [ ]:
# PaddleOCR 객체 모델 선언
ocr = PaddleOCR(lang='korean', max_text_length=max_text_length, drop_score=drop_score) # need to run only once to download and load model into memory

In [ ]:
# 추론
result = ocr.ocr(img_path, cls=False)


In [ ]:
texts = []
for i in range(len(result[0])):
    text = result[0][i][1][0]
    texts.append(text)
filtered_words = [word for word in texts if re.fullmatch(r'[가-힣]{3,4}', word)]
# 리스트를 하나의 문자열로 변환
filtered_words = set(filtered_words)
combined_string = ' '.join(filtered_words)